# Реализация автоматического дифференцирования

In [1]:
import tensorflow as tf

## 01 example

In [2]:
# создаем переменную
x = tf.Variable(-2.0)

with tf.GradientTape() as tape:
    y = x ** 2

df = tape.gradient(y, x)
print(df)    

tf.Tensor(-4.0, shape=(), dtype=float32)


## 02 exapmle

In [3]:
w = tf.Variable(tf.random.normal((3, 2)))
b = tf.Variable(tf.zeros(2, dtype=tf.float32))
x = tf.Variable([[-2.0, 1.0, 3.0]])

In [4]:
w

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.7462019 , -0.00618961],
       [ 1.2371953 , -1.3333826 ],
       [-0.9515552 , -0.8407472 ]], dtype=float32)>

In [5]:
b

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [6]:
x

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float32, numpy=array([[-2.,  1.,  3.]], dtype=float32)>

In [7]:
with tf.GradientTape() as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y ** 2)

df = tape.gradient(loss, [w, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(
[[  6.219748   7.68649 ]
 [ -3.109874  -3.843245]
 [ -9.329622 -11.529736]], shape=(3, 2), dtype=float32)
tf.Tensor([-3.109874 -3.843245], shape=(2,), dtype=float32)


В методе `gradient()` первым параметром указываем функцию `loss`, а вторым - список аргументов, от которых вычисляем частные производные в точках `w, b` и при заданном значении

## 03 example

In [8]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.constant(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
None


Переменная `b` - это константа, подтому производная по `b` не была вычислена

Преобразуе константу в переменную:

In [9]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


Запретим отслеживание переменной `x`:

In [10]:
x = tf.Variable(0, dtype=tf.float32, trainable=False)
b = tf.Variable(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

None
tf.Tensor(5.0, shape=(), dtype=float32)


Полное отключение отслеживания переменных:

In [11]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

None
None


Укажем переменные, которые следует наблюдать:

In [12]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
None


In [13]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch([x, b])
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


## 04 example

Также отслеживание происходит и для всех промежуточных переменных, которые связаны с наблюдаемой переменной.

In [14]:
x = tf.Variable(2, dtype=tf.float32)
y = tf.Variable(3, dtype=tf.float32)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y = 2 * x
    f = y * y
 
df = tape.gradient(f, y)
print(df)

tf.Tensor(8.0, shape=(), dtype=float32)
